# Pequeño tutorial de Pytorch

En este notebook vamos a desarrollar algunos aspectos básicos de PyTorch para implementar modelos sencillos

In [2]:
import torch
import numpy as np

Al igual que en Numpy, donde tenemos un objeto fundamental (ndarray), pytorch tiene su propio objeto, que es muy similar a un array de Numpy, pero con mayores funcionalidades, fundamentalmente relacionadas al producto de tensores y calculo de gradientes (autograd).

In [3]:
a = torch.zeros(3,2) # Tensor de ceros
b = torch.randn(3,3,2) # tensor de numeros aleatorios
print(a)
print(b)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[[ 0.9365, -0.7600],
         [ 0.5542,  1.6057],
         [-0.5896,  0.5641]],

        [[ 0.0469,  0.9249],
         [ 0.5157,  0.5632],
         [-0.7182, -1.3825]],

        [[ 0.8840, -1.0303],
         [-0.4446, -0.2949],
         [-0.2217, -0.7158]]])


In [4]:
c = torch.tensor([[1,2],[3,4],[5,6]]) # tensor a partir de una lista
print(c)

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [5]:
rng = np.random.default_rng()
a = rng.normal(size=(2,2,3))
b = torch.from_numpy(a)
print(b)    

tensor([[[ 0.4361, -0.7766,  0.6551],
         [ 1.2985, -2.0382, -0.4470]],

        [[ 0.5933,  0.6614,  0.7200],
         [-1.3538,  1.8248,  0.6804]]], dtype=torch.float64)


Las operaciones tipicas de numpy se pueden utilizar en pytorch: manejo de indices, operaciones aritmeticas

In [6]:
print(b)
b.shape

tensor([[[ 0.4361, -0.7766,  0.6551],
         [ 1.2985, -2.0382, -0.4470]],

        [[ 0.5933,  0.6614,  0.7200],
         [-1.3538,  1.8248,  0.6804]]], dtype=torch.float64)


torch.Size([2, 2, 3])

In [7]:
# En vez de reshape, se utiliza el metodo view
b.view(4,3)

tensor([[ 0.4361, -0.7766,  0.6551],
        [ 1.2985, -2.0382, -0.4470],
        [ 0.5933,  0.6614,  0.7200],
        [-1.3538,  1.8248,  0.6804]], dtype=torch.float64)

In [8]:
# al igual que en Numpy, -1 es un comodín para redimensionar
b.view(6,-1)

tensor([[ 0.4361, -0.7766],
        [ 0.6551,  1.2985],
        [-2.0382, -0.4470],
        [ 0.5933,  0.6614],
        [ 0.7200, -1.3538],
        [ 1.8248,  0.6804]], dtype=torch.float64)

In [9]:
b.view(-1)

tensor([ 0.4361, -0.7766,  0.6551,  1.2985, -2.0382, -0.4470,  0.5933,  0.6614,
         0.7200, -1.3538,  1.8248,  0.6804], dtype=torch.float64)

In [10]:
# Para hacer un cast de un tensor en un ndarray, basta con utilizar el metodo numpy()
b.numpy()

array([[[ 0.4360771 , -0.77658275,  0.65506992],
        [ 1.29853472, -2.03820208, -0.4469787 ]],

       [[ 0.59326631,  0.66141916,  0.72000364],
        [-1.35378468,  1.82475483,  0.6804384 ]]])

## Autograd - Grafos computacionales y diferenciacion!

In [11]:
x = torch.tensor(1., requires_grad= True) # requires_grad significa que guarda y calcula el grafo computacional
y = torch.tensor(2.,requires_grad= True)
f = x + y  # Tensor con la suma
z = torch.tensor(3., requires_grad= True)
g = f*z

In [12]:
g.backward() # el metodo backward recorre el grafo hacia atras encontrando todas las derivadas

Las derivadas se calculan llamando simplemente llamando al atributo grad

$$
\frac{dg}{dx} = \frac{dg}{df}\frac{df}{dx},\ \frac{dg}{dy} = \frac{dg}{df}\frac{df}{dy}, \frac{dg}{dz} = \frac{dg}{dz} 
$$
$$
\frac{dg}{dx} = c \times 1,\ \frac{dg}{dy} = z \times 1, \frac{dg}{dz} = f 
$$

In [15]:
print(x.grad, y.grad, z.grad)

tensor(3.) tensor(3.) tensor(3.)


## GPUS  - manejo

In [16]:
# Para saber si tenemos una GPU disponible
torch.cuda.is_available()

False

In [17]:
# En caso de tener una GPU disponible, basta con alojar los tensores en la GPU y operar con ellos
a = torch.randn(1000,1000).cuda() 

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [19]:
# una buena practica es definir una variable que identifique el tipo de procesador y utilice la gpu por defecto
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# y utilizar el device 
a = torch.tensor(1., requires_grad= True, device= device) # como no disponemos de gpu, se alojara en la cpu